In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sat-img-imgemb/emb_img/convNet_embeddings_id (1).pt
/kaggle/input/sat-img-imgemb/emb_img/efficient_embeddings_id.pt
/kaggle/input/sat-img-imgemb/emb_img/test_ConvNet_embeddings_id.pt
/kaggle/input/sat-img-imgemb/emb_img/test_resnet_id_embeddings.pt
/kaggle/input/sat-img-imgemb/emb_img/test_efficient_embeddings_by_id.pt
/kaggle/input/sat-img-imgemb/emb_img/convNet_embeddings_id.pt
/kaggle/input/sat-img-preprocessed-dataset/preprocessed_dataset/preprocessed_test.csv
/kaggle/input/sat-img-preprocessed-dataset/preprocessed_dataset/preprocessed1.csv
/kaggle/input/res-eff-emb/emb/res_embeddings_id.pt
/kaggle/input/res-eff-emb/emb/efficient_embeddings_idnew.pt
/kaggle/input/res-eff-emb/emb/test_efficient_embeddings_by_idnew.pt
/kaggle/input/res-eff-emb/emb/test_resnet_id_embeddings.pt


In [2]:
full_df=pd.read_csv("/kaggle/input/sat-img-preprocessed-dataset/preprocessed_dataset/preprocessed1.csv")

In [3]:
dft=pd.read_csv("/kaggle/input/sat-img-preprocessed-dataset/preprocessed_dataset/preprocessed_test.csv")

In [4]:
print(dft.shape)
dft.head()

(5404, 20)


,id,bedrooms,sqft_living,floors,waterfront,grade,zipcode,basement_ratio,above_ratio,rlt_size,rlt_lot_size,quality_liv,quality_cond,view_flag,water_view,luxury_bool,lot_util,bath_per_bed,house_life,ren_age
0,2591820310,4,2070,2.0,0,8,98058,0.000000,1.000000,0.866109,0.767761,16560,32,0,0,0,0.232767,0.562500,28,28
1,7974200820,5,2900,1.0,0,8,98115,0.368966,0.631034,1.223629,0.728101,23200,40,0,0,0,0.430906,0.600000,37,37
2,7701450110,4,3770,2.0,0,11,98006,0.000000,1.000000,1.016173,0.753644,41470,33,0,0,0,0.346094,0.625000,17,17
3,9522300010,3,4560,2.0,0,12,98034,0.000000,1.000000,1.125926,0.706484,54720,36,0,0,0,0.312158,1.166667,25,25
4,9510861140,3,2550,2.0,0,9,98052,0.000000,1.000000,1.133333,0.844755,22950,27,0,0,0,0.474330,0.833333,10,10


In [5]:
print(full_df.shape)
full_df.head()


(16209, 21)


,id,price,bedrooms,sqft_living,floors,waterfront,grade,zipcode,basement_ratio,above_ratio,...,rlt_lot_size,quality_liv,quality_cond,view_flag,water_view,luxury_bool,lot_util,bath_per_bed,house_life,ren_age
0,9117000170,268643,4,1810,2.0,0,7,98055,0.000000,1.000000,...,0.693147,12670,21,0,0,0,0.195887,0.562500,54,54
1,6700390210,245000,3,1600,2.0,0,7,98031,0.000000,1.000000,...,0.572882,11200,28,0,0,0,0.573888,0.833333,22,22
2,7212660540,200000,4,1720,2.0,0,8,98003,0.000000,1.000000,...,0.769499,13760,24,0,0,0,0.199120,0.625000,21,21
3,8562780200,352499,2,1240,2.0,0,7,98027,0.072581,0.927419,...,0.662688,8680,21,0,0,0,1.000000,1.125000,6,6
4,7760400350,232000,3,1280,1.0,0,7,98042,0.000000,1.000000,...,0.976374,8960,21,0,0,0,0.095837,0.666667,20,20


In [6]:
full_df=full_df.drop_duplicates(subset="id")
full_df.shape

(16110, 21)

In [7]:
dft=dft.drop_duplicates(subset="id")
dft.shape

(5396, 20)

In [8]:
from sklearn.model_selection import train_test_split

train_ids, val_ids = train_test_split(
    full_df["id"].unique(),
    test_size=0.2,
    random_state=42,   
    shuffle=True
)


In [9]:
import torch
import torch.nn as nn


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
full_df.columns

Index(['id', 'price', 'bedrooms', 'sqft_living', 'floors', 'waterfront',
       'grade', 'zipcode', 'basement_ratio', 'above_ratio', 'rlt_size',
       'rlt_lot_size', 'quality_liv', 'quality_cond', 'view_flag',
       'water_view', 'luxury_bool', 'lot_util', 'bath_per_bed', 'house_life',
       'ren_age'],
      dtype='object')

In [12]:
y=full_df['price']

In [13]:
y_log=np.log1p(y)

In [14]:
num_feature=[ 'price','bedrooms', 'sqft_living', 'floors', 'waterfront',
       'grade', 'zipcode', 'basement_ratio', 'above_ratio', 'rlt_size',
       'rlt_lot_size', 'quality_liv', 'quality_cond', 'view_flag',
       'water_view', 'luxury_bool', 'lot_util', 'bath_per_bed', 'house_life',
       'ren_age']

In [15]:
for col in full_df.columns:
    full_df[col] = pd.to_numeric(full_df[col], errors="coerce").astype(float)


In [16]:
for col in dft.columns:
    dft[col] = pd.to_numeric(dft[col], errors="coerce").astype(float)


In [17]:
full_df.head()

,id,price,bedrooms,sqft_living,floors,waterfront,grade,zipcode,basement_ratio,above_ratio,...,rlt_lot_size,quality_liv,quality_cond,view_flag,water_view,luxury_bool,lot_util,bath_per_bed,house_life,ren_age
0,9.117000e+09,268643.0,4.0,1810.0,2.0,0.0,7.0,98055.0,0.000000,1.000000,...,0.693147,12670.0,21.0,0.0,0.0,0.0,0.195887,0.562500,54.0,54.0
1,6.700390e+09,245000.0,3.0,1600.0,2.0,0.0,7.0,98031.0,0.000000,1.000000,...,0.572882,11200.0,28.0,0.0,0.0,0.0,0.573888,0.833333,22.0,22.0
2,7.212661e+09,200000.0,4.0,1720.0,2.0,0.0,8.0,98003.0,0.000000,1.000000,...,0.769499,13760.0,24.0,0.0,0.0,0.0,0.199120,0.625000,21.0,21.0
3,8.562780e+09,352499.0,2.0,1240.0,2.0,0.0,7.0,98027.0,0.072581,0.927419,...,0.662688,8680.0,21.0,0.0,0.0,0.0,1.000000,1.125000,6.0,6.0
4,7.760400e+09,232000.0,3.0,1280.0,1.0,0.0,7.0,98042.0,0.000000,1.000000,...,0.976374,8960.0,21.0,0.0,0.0,0.0,0.095837,0.666667,20.0,20.0


In [25]:
X_tab=full_df[num_feature]

In [19]:
numt_feature=['bedrooms', 'sqft_living', 'floors', 'waterfront',
       'grade', 'zipcode', 'basement_ratio', 'above_ratio', 'rlt_size',
       'rlt_lot_size', 'quality_liv', 'quality_cond', 'view_flag',
       'water_view', 'luxury_bool', 'lot_util', 'bath_per_bed', 'house_life',
       'ren_age']

In [45]:
test_ids = dft["id"]

In [20]:
Xt_tab=dft[numt_feature]

In [111]:
X_tab1=X_tab[numt_feature]

In [26]:
X_tab.shape

(16110, 20)

In [27]:
Xt_tab.shape

(5396, 19)

# Tabular Only Training

In [28]:
train_df = X_tab[full_df["id"].isin(train_ids)]
val_df   = X_tab[full_df["id"].isin(val_ids)]

In [29]:
target_col = "price"

X_train = train_df.drop(columns=[target_col])
y_train = train_df[target_col]
y_train_log=np.log1p(y_train)

X_val = val_df.drop(columns=[target_col])
y_val = val_df[target_col]
y_val_log=np.log1p(y_val)

In [30]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

import lightgbm as lgb

/usr/local/lib/python3.12/dist-packages/sqlalchemy/orm/query.py:195: SyntaxWarning: "is not" with 'tuple' literal. Did you mean "!="?
  if entities is not ():


In [31]:
cat_cols = ["zipcode"]
num_cols = [c for c in num_feature if c not in (cat_cols+["price"])]


In [32]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
    ]
)


In [33]:
pipe = Pipeline([
    ("prep", preprocessor),
    ("model", lgb.LGBMRegressor(
        objective="regression",
        n_estimators=500,
        learning_rate=0.05,
        random_state=42
    ))
])


In [34]:
pipe.fit(X_train, y_train_log)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002449 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2277
[LightGBM] [Info] Number of data points in the train set: 12888, number of used features: 88
[LightGBM] [Info] Start training from score 13.047016


Pipeline(steps=[('prep',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['bedrooms', 'sqft_living',
                                                   'floors', 'waterfront',
                                                   'grade', 'basement_ratio',
                                                   'above_ratio', 'rlt_size',
                                                   'rlt_lot_size',
                                                   'quality_liv',
                                                   'quality_cond', 'view_flag',
                                                   'water_view', 'luxury_bool',
                                                   'lot_util', 'bath_per_bed',
                                                   'house_life', 'ren_age']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['zipcode'])])),
                ('model',
                 LGBMRegressor(learning_rate=0.05, n_estimators=500,
                               objective='regression', random_state=42))])

In [36]:
val_preds_tab_log = pipe.predict(X_val)
val_preds_tab=np.expm1(val_preds_log)


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [37]:
from sklearn.metrics import r2_score, mean_squared_error

In [38]:
r2_lgb=r2_score(y_val_log,val_preds_tab_log)
mse_lgb=mean_squared_error(y_val_log,val_preds_tab_log)

print("r2 score: ",r2_lgb)
print("mse error: ",mse_lgb)
print()
print("REAL:")

r2_lgb=r2_score(y_val,val_preds_tab)
mse_lgb=mean_squared_error(y_val,val_preds_tab)

print("r2 score: ",r2_lgb)
print("mse error: ",mse_lgb)

r2 score:  0.8855230032470994
mse error:  0.032031236724609904

REAL:
r2 score:  0.8837937492229008
mse error:  14205234010.705217


In [39]:
test_preds_tab=pipe.predict(Xt_tab)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [47]:
test_preds_tab=np.expm1(test_preds_tab)


In [48]:
submission = pd.DataFrame({
    "id": test_ids,
    "price": test_preds_tab
})

submission.to_csv("only_tabular.csv", index=False)

In [50]:
val_preds_log_lgb=val_preds_log
val_preds_lgb=val_preds

# Image Model Train

In [51]:
id_to_idx = {int(idv): i for i, idv in enumerate(full_df["id"].values)}
train_idx_torch = torch.tensor(
    [id_to_idx[int(i)] for i in train_ids],
    dtype=torch.long
)
val_idx_torch = torch.tensor(
    [id_to_idx[int(i)] for i in val_ids],
    dtype=torch.long
)

In [52]:
train_idx=train_idx_torch.cpu().numpy()
val_idx=val_idx_torch.cpu().numpy()

In [53]:
Conv_emb_dict = torch.load("/kaggle/input/sat-img-imgemb/emb_img/convNet_embeddings_id.pt")


In [54]:
ids = full_df["id"].values

Conv_img = torch.stack([Conv_emb_dict[int(i)] for i in ids])



In [55]:
X_Conv = Conv_img.cpu().numpy()


In [56]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


# 1️⃣ Standardize (MANDATORY before PCA)
scaler = StandardScaler()
X_Conv_scaled = scaler.fit_transform(X_Conv)

# 2️⃣ PCA reduction
pca = PCA(n_components=256, random_state=42)
X_Conv_pca = pca.fit_transform(X_Conv_scaled)

print("Reduced shape:", X_Conv_pca.shape)

# (optional) save for later use
import joblib
joblib.dump(pca, "/kaggle/working/convnext_pca.pkl")
joblib.dump(scaler, "/kaggle/working/convnext_scaler.pkl")


Reduced shape: (16110, 256)


['/kaggle/working/convnext_scaler.pkl']

In [57]:
X_Conv_pca.shape

(16110, 256)

In [58]:
X_train_Conv = X_Conv_pca[train_idx]
X_val_Conv   = X_Conv_pca[val_idx]

y_train_log = y_log.iloc[train_idx].values if hasattr(y_log, "iloc") else y_log[train_idx]
y_val_log   = y_log.iloc[val_idx].values if hasattr(y_log, "iloc") else y_log[val_idx]


In [59]:

lgb_model_Conv = lgb.LGBMRegressor(
    objective="regression",
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)

In [60]:
lgb_model_Conv.fit(
    X_train_Conv,
    y_train_log,
    eval_set=[(X_val_Conv, y_val_log)],
    eval_metric="rmse",
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(100)
    ]
)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.021913 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 65280
[LightGBM] [Info] Number of data points in the train set: 12888, number of used features: 256
[LightGBM] [Info] Start training from score 13.047016
Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.375183	valid_0's l2: 0.140762
[200]	valid_0's rmse: 0.372066	valid_0's l2: 0.138433
Early stopping, best iteration is:
[200]	valid_0's rmse: 0.372066	valid_0's l2: 0.138433


LGBMRegressor(learning_rate=0.05, n_estimators=500, objective='regression',
              random_state=42)

In [61]:
val_preds_Conv_log = lgb_model_Conv.predict(X_val_Conv)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [62]:
val_preds_Conv=np.expm1(val_preds_Conv_log)

In [63]:
y_val=y.iloc[val_idx]

In [64]:
r2_lgb=r2_score(y_val_log,val_preds_Conv_log)
mse_lgb=mean_squared_error(y_val_log,val_preds_Conv_log)

print("r2 score: ",r2_lgb)
print("mse error: ",mse_lgb)
print()
print("REAL:")

r2_lgb=r2_score(y_val,val_preds_Conv)
mse_lgb=mean_squared_error(y_val,val_preds_Conv)

print("r2 score: ",r2_lgb)
print("mse error: ",mse_lgb)

r2 score:  0.5052504450334833
mse error:  0.13843340202866006

REAL:
r2 score:  0.43845440624654497
mse error:  68644212455.05187


# Late Fusion

In [65]:
from sklearn.linear_model import Ridge

X_stack = np.vstack([val_preds, val_preds_Conv]).T

meta = Ridge(alpha=1.0)
meta.fit(X_stack, y_val)

y_stack = meta.predict(X_stack)

mse = mean_squared_error(y_val, y_stack)
r2 = r2_score(y_val, y_stack)

print("Late Fusion (stacking) MSE:", mse)
print("Late Fusion (stacking) R²:", r2)


Late Fusion (stacking) MSE: 67402871369.013145
Late Fusion (stacking) R²: 0.4486092261837191


# Efficient Net

In [67]:
Eff_emb_dict=torch.load("/kaggle/input/res-eff-emb/emb/efficient_embeddings_idnew.pt")

In [68]:
Eff_img = torch.stack([Eff_emb_dict[int(i)] for i in ids])


In [69]:
X_eff=Eff_img.cpu().numpy()

In [70]:
X_eff.shape

(16110, 1280)

In [71]:
scaler_eff = StandardScaler()
X_eff_scaled = scaler_eff.fit_transform(X_eff)

pca_eff = PCA(n_components=256, random_state=42)
X_eff_pca = pca_eff.fit_transform(X_eff_scaled)

print("Reduced shape:", X_eff_pca.shape)

joblib.dump(pca_eff, "/kaggle/working/efficientnet_pca.pkl")
joblib.dump(scaler_eff, "/kaggle/working/efficientnet_scaler.pkl")


Reduced shape: (16110, 256)


['/kaggle/working/efficientnet_scaler.pkl']

In [72]:
X_train_eff = X_eff_pca[train_idx]
X_val_eff   = X_eff_pca[val_idx]

#y_train_log = y_log.iloc[train_idx].values if hasattr(y, "iloc") else y[train_idx]
#y_val_log   = y_log.iloc[val_idx].values if hasattr(y, "iloc") else y[val_idx]


In [73]:
lgb_model_eff = lgb.LGBMRegressor(
    objective="regression",
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)

In [74]:
lgb_model_eff.fit(
    X_train_eff,
    y_train_log,
    eval_set=[(X_val_Conv, y_val_log)],
    eval_metric="rmse",
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(100)
    ]
)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019139 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 65280
[LightGBM] [Info] Number of data points in the train set: 12888, number of used features: 256
[LightGBM] [Info] Start training from score 13.047016
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[1]	valid_0's rmse: 0.533758	valid_0's l2: 0.284897


LGBMRegressor(learning_rate=0.05, n_estimators=500, objective='regression',
              random_state=42)

In [75]:
val_preds_eff_log = lgb_model_eff.predict(X_val_eff)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [76]:
val_preds_eff=np.expm1(val_preds_eff_log)

In [77]:
r2_lgb=r2_score(y_val_log,val_preds_eff_log)
mse_lgb=mean_squared_error(y_val_log,val_preds_eff_log)

print("r2 score: ",r2_lgb)
print("mse error: ",mse_lgb)
print()
print("REAL:")

r2_lgb=r2_score(y_val,val_preds_eff)
mse_lgb=mean_squared_error(y_val,val_preds_eff)

print("r2 score: ",r2_lgb)
print("mse error: ",mse_lgb)

r2 score:  0.02500055970883308
mse error:  0.27280972391108105

REAL:
r2 score:  -0.029437861817831035
mse error:  125840095767.75838


# Resnet

In [78]:
res_emb_dict=torch.load("/kaggle/input/res-eff-emb/emb/res_embeddings_id.pt")

In [79]:
res_img = torch.stack([res_emb_dict[int(i)] for i in ids])

In [80]:
X_res=res_img.cpu().numpy()
X_res.shape

(16110, 2048)

In [81]:
scaler_res = StandardScaler()
X_res_scaled = scaler_res.fit_transform(X_res)

pca_res = PCA(n_components=256, random_state=42)
X_res_pca = pca_res.fit_transform(X_res_scaled)

print("Reduced shape:", X_res_pca.shape)

joblib.dump(pca_res, "/kaggle/working/efficientnet_pca.pkl")
joblib.dump(scaler_res, "/kaggle/working/efficientnet_scaler.pkl")


Reduced shape: (16110, 256)


['/kaggle/working/efficientnet_scaler.pkl']

In [82]:
X_train_res = X_res_pca[train_idx]
X_val_res   = X_res_pca[val_idx]

y_train_log = y_log.iloc[train_idx].values if hasattr(y_log, "iloc") else y_log[train_idx]
y_val_log   = y_log.iloc[val_idx].values if hasattr(y_log, "iloc") else y_log[val_idx]


### ...................................... resnet only prediction .....................................

In [83]:
lgb_model_res = lgb.LGBMRegressor(
    objective="regression",
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)

In [84]:


lgb_model_res.fit(
    X_train_res,
    y_train_log,
    eval_set=[(X_val_res, y_val_log)],
    eval_metric="rmse",
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(100)
    ]
)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019388 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 65280
[LightGBM] [Info] Number of data points in the train set: 12888, number of used features: 256
[LightGBM] [Info] Start training from score 13.047016
Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.423495	valid_0's l2: 0.179348
[200]	valid_0's rmse: 0.414369	valid_0's l2: 0.171702
[300]	valid_0's rmse: 0.412649	valid_0's l2: 0.170279
[400]	valid_0's rmse: 0.41165	valid_0's l2: 0.169456
[500]	valid_0's rmse: 0.41109	valid_0's l2: 0.168995
Did not meet early stopping. Best iteration is:
[500]	valid_0's rmse: 0.41109	valid_0's l2: 0.168995


LGBMRegressor(learning_rate=0.05, n_estimators=500, objective='regression',
              random_state=42)

In [85]:
val_preds_res_log = lgb_model_res.predict(X_val_res)
val_preds_res=np.expm1(val_preds_res_log)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [86]:
r2_lgb=r2_score(y_val_log,val_preds_res_log)
mse_lgb=mean_squared_error(y_val_log,val_preds_res_log)

print("r2 score: ",r2_lgb)
print("mse error: ",mse_lgb)
print()
print("REAL:")

r2_lgb=r2_score(y_val,val_preds_res)
mse_lgb=mean_squared_error(y_val,val_preds_res)

print("r2 score: ",r2_lgb)
print("mse error: ",mse_lgb)

r2 score:  0.3960267053516161
mse error:  0.16899475112876572

REAL:
r2 score:  0.2723984973679683
mse error:  88943146709.50237


## Combining Images

In [89]:
rest_emb_dict=torch.load("/kaggle/input/res-eff-emb/emb/test_resnet_id_embeddings.pt")

In [91]:
Efft_emb_dict=torch.load("/kaggle/input/res-eff-emb/emb/test_efficient_embeddings_by_idnew.pt")

In [90]:
Convt_emb_dict=torch.load("/kaggle/input/sat-img-imgemb/emb_img/test_ConvNet_embeddings_id.pt")

In [95]:
X_img = torch.cat([
    torch.stack([res_emb_dict[int(i)] for i in ids]),
       torch.stack([Eff_emb_dict[int(i)] for i in ids]),
    torch.stack([Conv_emb_dict[int(i)] for i in ids]),
], dim=1)

In [93]:
Xt_img = torch.cat([
    torch.stack([rest_emb_dict[int(i)] for i in test_ids]),
       torch.stack([Efft_emb_dict[int(i)] for i in test_ids]),
    torch.stack([Convt_emb_dict[int(i)] for i in test_ids]),
], dim=1)

In [96]:
scaler_img = StandardScaler()
X_img_scaled = scaler_img.fit_transform(X_img)

pca_img = PCA(n_components=256, random_state=42)
X_img_pca = pca_img.fit_transform(X_img_scaled)

print("Reduced shape:", X_img_pca.shape)


Reduced shape: (16110, 256)


In [97]:
scaler_imgt = StandardScaler()
Xt_img_scaled = scaler_imgt.fit_transform(Xt_img)

pca_imgt = PCA(n_components=256, random_state=42)
Xt_img_pca = pca_imgt.fit_transform(Xt_img_scaled)

print("Reduced shape:", Xt_img_pca.shape)


Reduced shape: (5396, 256)


### ...........................................Image Models Only Prediction.............................

In [98]:
X_train_img = X_img_pca[train_idx]
X_val_img  = X_img_pca[val_idx]

y_train_log = y_log.iloc[train_idx].values if hasattr(y_log, "iloc") else y_log[train_idx]
y_val_log   = y_log.iloc[val_idx].values if hasattr(y_log, "iloc") else y_log[val_idx]


In [99]:
lgb_model_img = lgb.LGBMRegressor(
    objective="regression",
    n_estimators=500,
    learning_rate=0.05,
    num_leaves=31,
    random_state=42
)

In [100]:


lgb_model_img.fit(
    X_train_img,
    y_train_log,
    eval_set=[(X_val_img, y_val_log)],
    eval_metric="rmse",
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(100)
    ]
)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019582 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 65280
[LightGBM] [Info] Number of data points in the train set: 12888, number of used features: 256
[LightGBM] [Info] Start training from score 13.047016
Training until validation scores don't improve for 50 rounds
[100]	valid_0's rmse: 0.372068	valid_0's l2: 0.138435
[200]	valid_0's rmse: 0.369267	valid_0's l2: 0.136358
[300]	valid_0's rmse: 0.368688	valid_0's l2: 0.135931
Early stopping, best iteration is:
[337]	valid_0's rmse: 0.36837	valid_0's l2: 0.135696


LGBMRegressor(learning_rate=0.05, n_estimators=500, objective='regression',
              random_state=42)

In [101]:
val_preds_img_log = lgb_model_img.predict(X_val_img)
val_preds_img=np.expm1(val_preds_img_log)

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [102]:
r2_lgb=r2_score(y_val_log,val_preds_img_log)
mse_lgb=mean_squared_error(y_val_log,val_preds_img_log)

print("Score for log(1+y) as target:")
print("r2 score: ",r2_lgb)
print("mse error: ",mse_lgb)
print()
print("REAL:")

r2_lgb=r2_score(y_val,val_preds_img)
mse_lgb=mean_squared_error(y_val,val_preds_img)

print("r2 score: ",r2_lgb)
print("mse error: ",mse_lgb)

Score for log(1+y) as target:
r2 score:  0.5150320504363513
mse error:  0.13569645987352752

REAL:
r2 score:  0.4559296920037358
mse error:  66508006167.31093


### .............................Late Fusion only for Image models....................................

In [103]:
X_stack_img = np.vstack([
    val_preds_Conv,
    val_preds_eff,
    val_preds_res
]).T


In [104]:
from sklearn.linear_model import Ridge
meta = Ridge()
meta.fit(X_stack_img, y_val)


Ridge()

In [105]:
y_stack = meta.predict(X_stack_img)

mse = mean_squared_error(y_val, y_stack)
r2 = r2_score(y_val, y_stack)

print("Late Fusion (stacking) MSE:", mse)
print("Late Fusion (stacking) R²:", r2)


Late Fusion (stacking) MSE: 65062016013.0889
Late Fusion (stacking) R²: 0.4677586484543026


# Fusion

# Mid Level Fusion

In [112]:
X_tab1.head()
X_tab1.shape

(16110, 19)

In [113]:
print(Xt_img_pca.shape)
Xt_tab.shape

(5396, 256)


(5396, 19)

In [114]:
print(X_img_pca.shape)
X_tab1.shape

(16110, 256)


(16110, 19)

In [115]:
X_fused = np.concatenate([X_tab1, X_img_pca], axis=1)


In [116]:
Xt_fused = np.concatenate([Xt_tab, Xt_img_pca], axis=1)


In [117]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, shuffle=True, random_state=42)

oof_pred = np.zeros(len(y))
models = []


In [122]:
import xgboost as xgb

In [126]:
params = {
    "objective": "reg:squarederror",
    "learning_rate": 0.05,
    "max_depth": 6,
    "subsample": 0.8,
    "colsample_bytree": 0.8,
    "n_estimators": 2000,
    "random_state": 42,
    "tree_method": "hist",   # valid option
    "device": "cuda"         # enable GPU if available
}


In [127]:
y_log.size

16110

In [128]:

for fold, (tr, val) in enumerate(kf.split(X_fused)):
    print(f"Fold {fold+1}")

    X_tr, X_val = X_fused[tr], X_fused[val]

    # correct y indexing (NO train_idx / val_idx here)
    y_tr = y_log.iloc[tr].values if hasattr(y_log, "iloc") else y_log[tr]
    y_val = y_log.iloc[val].values if hasattr(y_log, "iloc") else y_log[val]

    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    dval   = xgb.DMatrix(X_val, label=y_val)

    model = xgb.train(
        params,
        dtrain,
        num_boost_round=2000,
        evals=[(dval, "val")],
        early_stopping_rounds=100,
        verbose_eval=False
    )

    oof_pred[val] = model.predict(dval)
    models.append(model)


Fold 1


/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [17:32:17] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


Fold 2


/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [17:32:26] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


Fold 3


/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [17:32:34] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


Fold 4


/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [17:32:42] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


Fold 5


/usr/local/lib/python3.12/dist-packages/xgboost/callback.py:386: UserWarning: [17:32:51] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "n_estimators" } are not used.

  self.starting_round = model.num_boosted_rounds()


In [129]:
oof_pred = np.clip(oof_pred, -10, 15)

y_pred_price = np.expm1(oof_pred)

mse = mean_squared_error(y, y_pred_price)
r2 = r2_score(y, y_pred_price)

print("XGBoost Mid-Fusion CV MSE:", mse)
print("XGBoost Mid-Fusion CV R²:", r2)


XGBoost Mid-Fusion CV MSE: 22160504014.048695
XGBoost Mid-Fusion CV R²: 0.829339103663111


In [130]:
# X_test_fused: [N_test, D]
dtest = xgb.DMatrix(Xt_fused)


In [131]:
test_preds_log = []

for model in models:
    pred = model.predict(dtest)
    test_preds_log.append(pred)


In [132]:
test_preds_log = np.mean(test_preds_log, axis=0)

In [133]:
test_preds_log = np.clip(test_preds_log, -10, 15)
test_preds_price = np.expm1(test_preds_log)

In [134]:
test_preds_price

array([ 406316.94,  724641.7 , 1258722.2 , ...,  319740.8 , 1418802.1 ,
        349793.62], dtype=float32)

In [135]:
submission = pd.DataFrame({
    "id": test_ids,
    "price": test_preds_price
})

submission.to_csv("img_and_tab.csv", index=False)